# 영화 추천 및 즐겨찾기 GUI
### 조건에 따른 영화 선택 및 즐겨찾기 

### 1. 데이터 처리

In [113]:
# 데이터 소개
# kaggle https://www.kaggle.com/code/ayushimishra2809/movie-recommendation-system/input?select=ratings.csv 사용

In [114]:
import pandas as pd
from datetime import datetime
import re


In [115]:
# 데이터 불러오기
raw_movies = pd.read_csv('movies.csv')
raw_user_rating = pd.read_csv('ratings.csv')
print(raw_movies.head(1)) 
print(raw_user_rating.head(1))

   movieId             title                                       genres
0        1  Toy Story (1995)  Adventure|Animation|Children|Comedy|Fantasy
   userId  movieId  rating   timestamp
0       1       16     4.0  1217897793


In [116]:
#데이터 전처리

#영화 수 너무 많음. #100개만으로 하기
movies = raw_movies.sample(n = 100, random_state=42)
# 유저 별 영화 평점 평균
rating = raw_user_rating[['movieId', 'rating']].groupby('movieId').mean().round(1).reset_index()
rating.head(1)
# 합치기 (컬럼 : 영화 제목(날짜), 영화 장르, 평점)
data = movies.merge(rating)
data = data.drop('movieId', axis = 1)

In [117]:
# 제목에서 연도 지우기 +  연도는 따라 딕셔너리에 저장
# 장르 리스트 생성

movie_date = {}
genre = set()
for idx,i in data.iterrows():
    movie_date[i['title'][:-7]] = int(i['title'][-5:-1])
    data.at[idx, 'title'] = i['title'][:-7]
    for j in i['genres'].split('|'):
        genre.add(j)
genre = list(genre)

### GUI 만들기 시작

In [118]:
import tkinter as tk
from tkinter import *
from tkinter import ttk
from tkinter import messagebox


In [119]:
w = Tk()
w.title('시작')

def login(): #로그인
    id_f = open('User_Id.txt', 'r') #회원관리 파일에 존재하는 아이디, 비밀번호인지 체크
    id = id_f.read()
    pw_f = open('User_pw.txt', 'r')
    pw = pw_f.read()
    id_f.close()
    pw_f.close()

    if user_id.get() == id and password.get() == pw:
        print("Logged IN Successfully")
        messagebox.showinfo('로그인', '환영합니다.')
        start()
    else:
        messagebox.showinfo('로그인', '아이디/비밀번호가 맞지 않습니다')

def start(): #메인 화면
    global new1
    global movie_search
    new1 = Toplevel()
    new1.title('나만의 영화장')
    frame1 = Frame(new1)
    frame1.pack()
    l1 = Label(frame1, text = '원하는 영화를 검색해주세요').grid(row = 0,column=0)
    movie_search = Entry(frame1)
    movie_search.grid(row=1)
    butt1 = Button(frame1, text = '검색', command = search_movie, bg = 'blue', fg = 'white').grid(row=1,column=1)
    butt2 = Button(frame1, text = '장르선택', command = genre_select, bg = 'sky blue').grid(row=2, column=0)
    butt3 = Button(frame1, text = '즐겨찾기 보기', command = recommend_show, bg = 'sky blue').grid(row=3,column=0)

def search_movie(): # 영화 검색
    global new2
    new2 = Toplevel()
    new2.title('영화검색')
    frame2 = Frame(new2)
    frame2.pack()
    s_movie = movie_search.get()
    if s_movie in list(data['title']):
        row = data[data['title'] == s_movie].iloc[0]
        text = f"제목 : {row['title']}, 장르 : {row['genres']}, 평점 : {row['rating']}"
        butt1 = Button(frame2, text = text, command = lambda t=text: recommend_add(t), bg = 'bisque').pack()
    else:
        messagebox.showinfo('오류', '해당하는 영화가 없습니다.')
        new2.destroy()

def genre_select(): #장르 선택
    global new3
    global genre_combo
    new3 = Toplevel()
    new3.title('장르검색')
    frame3 = Frame(new3)
    frame3.pack()
    genre_combo = ttk.Combobox(frame3, values = genre)
    genre_combo.pack()
    butt1 = Button(frame3, text = '선택완료', command = genre_movie, bg = 'blue', fg = 'white').pack()

def genre_movie(): #장르로 영화 검색
    global new4
    global selected_genre
    global frame4
    new4 = Toplevel()
    new4.title('장르검색')
    frame4 = Frame(new4)
    frame4.pack()
    mainMenu = Menu(new4)
    new4.config(menu = mainMenu)
    fileMenu = Menu(mainMenu)
    mainMenu.add_cascade(label = "정렬", menu = fileMenu)
    fileMenu.add_command(label = '별점순', command = sort_rating)
    fileMenu.add_separator()
    fileMenu.add_command(label = '날짜순', command = sort_dating)

    selected_genre = genre_combo.get()
    for idx, i in data.iterrows():
        if selected_genre in i['genres']:
            text = f"제목 : {i['title']}, 장르 : {i['genres']}, 평점 : {i['rating']}"
            Button(frame4, text=text, command=lambda t=text: recommend_add(t), bg = 'bisque').pack(pady = 5)

def sort_rating(): #별점 정렬 (높은순)
    s_r_data = data.loc[[idx for idx, i in data.iterrows() if selected_genre in i['genres']]].sort_values(by = 'rating', ascending = False)
    for widget in frame4.winfo_children():
        widget.destroy()

    for idx, i in s_r_data.iterrows():
        text = f"제목 : {i['title']}, 장르 : {i['genres']}, 평점 : {i['rating']}"
        Button(frame4, text=text, command=lambda t=text: recommend_add(t), bg = 'bisque').pack()


def sort_dating(): #날짜 정렬 (최신순)
    date = data[:]
    d_list = [movie_date[j] for j in list(data['title'])]
    date['date'] = d_list
    date = date.loc[[idx for idx, i in date.iterrows() if selected_genre in i['genres']]].sort_values(by = 'date', ascending = False)
    for widget in frame4.winfo_children():
        widget.destroy()

    for idx, i in date.iterrows():
        text = f"제목 : {i['title']}, 장르 : {i['genres']}, 평점 : {i['rating']}"
        Button(frame4, text=text, command=lambda t=text: recommend_add(t), bg = 'bisque').pack()

def recommend_show(): #즐겨찾기 확인
    global new5
    new5 = Toplevel()
    new5.title('즐겨찾기')
    frame5 = Frame(new5)
    frame5.pack()
    f = open('recommend.txt', 'r')
    while True:
        line  = f.readline()
        if not line: break
        Label(frame5, text = line[:-1]).pack()
    f.close()

def recommend_add(content): #즐겨찾기 추가
    f = open('recommend.txt', 'a')
    f.write(content+'\n')
    f.close()
    print('추가완료')
    messagebox.showinfo('즐겨찾기', '즐겨찾기에 추가되었습니다^^')


messagebox.showinfo('시작', '나만의 영화장에 오신 걸 환영합니다.')
user_id, password = StringVar(), StringVar()
tk.Label(w, text = "Username : ").grid(row=1, column=0)
tk.Label(w, text = "Password : ").grid(row=2, column=0)
tk.Entry(w, textvariable = user_id).grid(row=1, column=1)
tk.Entry(w, show = '*', textvariable = password).grid(row=2, column=1)
tk.Button(w, text = "Login", command = login, bg="blue", fg="white").grid(row=3, column=0,columnspan=2)

mainloop()

Logged IN Successfully
추가완료


C:\Users\Jeonghwan Cho\AppData\Local\Temp\ipykernel_5212\1071287749.py:94: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  date['date'] = d_list


In [120]:
# 이번 학기 제게 정말 필요한 내용을 다루어주셔서 감사합니다. 덕분에 부족한 부분을 많이 보완할 수 있었습니다. 교수님, 조교님 모두 수고하셨습니다. 감사합니다.